# Conversational agent

In [ ]:
import json
import boto3
import panel as pn  # GUI

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

In [ ]:
from langchain.tools import tool

In [ ]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [ ]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [ ]:
tools = [get_current_temperature, search_wikipedia]

In [ ]:
from langchain_aws import ChatBedrock
from langchain.chains.api.prompt import API_URL_PROMPT
from langchain_community.agent_toolkits.openapi.toolkit import RequestsToolkit
from langchain_community.utilities.requests import TextRequestsWrapper
from langchain.agents.output_parsers import ToolsAgentOutputParser
from langchain.prompts import ChatPromptTemplate


In [ ]:
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0)
)
model_with_tools = model.bind_tools(tools=tools)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant. Use tools when possible. Make best guesses about city locations."),
    ("user", "{input}"),
])
chain = prompt | model_with_tools | ToolsAgentOutputParser()

In [ ]:
result = chain.invoke({"input": "what is the weather is sf?"})

In [ ]:
result[0].tool

In [ ]:
result[0].tool_input

In [ ]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant. Use tools when possible. Make best guesses about city locations."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
def finish_result(input):
    if isinstance(input, list) and len(input):
        return input[0]
    return input

In [ ]:
chain = prompt | model_with_tools | ToolsAgentOutputParser() | finish_result

In [ ]:
result1 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": []
})

In [ ]:
result1.tool

In [ ]:
observation = get_current_temperature.run(result1.tool_input)

In [ ]:
observation

In [ ]:
type(result1)

In [ ]:
from langchain.agents.format_scratchpad import format_to_tool_messages

In [ ]:
result1.message_log

In [ ]:
format_to_tool_messages([(result1, observation), ])

In [ ]:
result2 = chain.invoke({
    "input": "what is the weather is sf?", 
    "agent_scratchpad": format_to_tool_messages([(result1, observation), ])
})

In [ ]:
result2

In [ ]:
from langchain.schema.agent import AgentFinish
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input,
            "agent_scratchpad": format_to_tool_messages(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_tool_messages(x["intermediate_steps"])
) | chain

In [ ]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input,
            "intermediate_steps": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [ ]:
run_agent("what is the weather is sf?")

In [ ]:
run_agent("what is langchain?")

In [ ]:
run_agent("hi!")

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "what is langchain?"})

In [ ]:
agent_executor.invoke({"input": "my name is bob"})

In [ ]:
agent_executor.invoke({"input": "what is my name"})

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant. Use tools when possible. Make best guesses about city locations."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_tool_messages(x["intermediate_steps"])
) | prompt | model | ToolsAgentOutputParser()

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [ ]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [ ]:
agent_executor.invoke({"input": "my name is bob"})

In [ ]:
agent_executor.invoke({"input": "whats my name"})

In [ ]:
agent_executor.invoke({"input": "whats the weather in sf?"})

### Create a chatbot

In [ ]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [ ]:
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [ ]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        #self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = model.bind_tools(tools=tools)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant. Use tools when possible. Make best guesses about city locations."),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_tool_messages(x["intermediate_steps"])
        ) | self.prompt | self.model | ToolsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)

    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        print("result is a ", type(result))
        self.answer = result['output']
        if isinstance(self.answer, list) and len(self.answer):
            self.answer = self.answer[0]
        if isinstance(self.answer, dict) and self.answer["type"] == "text":
            self.answer = self.answer["text"]
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [ ]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard